In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import seaborn as sns
df= sns.load_dataset('tips')

In [5]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [7]:
df['sex'].value_counts()

Male      157
Female     87
Name: sex, dtype: int64

In [8]:
df['smoker'].value_counts()

No     151
Yes     93
Name: smoker, dtype: int64

In [9]:
df['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [10]:
df['time'].value_counts()

Dinner    176
Lunch      68
Name: time, dtype: int64

In [11]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [12]:
X = df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y = df['total_bill']

In [13]:
from sklearn.model_selection import train_test_split  
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
# Encoding categorical variables
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

leb1 = LabelEncoder()
leb2 = LabelEncoder()
leb3 = LabelEncoder()
X_train['sex']=leb1.fit_transform(X_train['sex'])
X_train['smoker']=leb2.fit_transform(X_train['smoker'])
X_train['time']=leb3.fit_transform(X_train['time'])



In [15]:

X_test['sex'] = leb1.transform(X_test['sex'])
X_test['smoker'] = leb2.transform(X_test['smoker'])
X_test['time'] = leb3.transform(X_test['time'])

In [16]:
## OneHot Encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct=ColumnTransformer(transformers=[('onehot', OneHotEncoder(drop='first'),[3])], remainder='passthrough')


In [17]:
X_train=ct.fit_transform(X_train)

In [18]:
X_test=ct.transform(X_test)

In [19]:
# Model Training
from sklearn.svm import SVR
svr = SVR()
svr.fit(X_train, y_train)


SVR()

In [20]:
y_pred=svr.predict(X_test)

In [21]:
from sklearn.metrics import mean_absolute_error, r2_score
mae=mean_absolute_error(y_test,y_pred)
R2Score=r2_score(y_test,y_pred)
print(f'Mean Absolute Error: {mae}')
print(f'R2 Score: {R2Score}')

Mean Absolute Error: 4.413774883497757
R2 Score: 0.5502250005312896


## I got 55% accuracy on SVR without hyperparameter tuning.


In [22]:
from sklearn.model_selection import GridSearchCV
param_grid = {
      'kernel': ['rbf'],
      'C': [0.1, 1, 10, 100, 1000],
      'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
}
grid_search = GridSearchCV(SVR(), param_grid, cv=5, verbose=3)
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.119 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.014 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.017 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.019 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.042 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.013 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.099 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.080 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.080 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.049 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.109 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}

In [23]:
y_pred=grid_search.predict(X_test)
mae=mean_absolute_error(y_test,y_pred)
R2Score=r2_score(y_test,y_pred)

In [24]:
print(f'Mean Absolute Error: {mae}')
print(f'R2 Score: {R2Score}')

Mean Absolute Error: 4.129773285984329
R2 Score: 0.6200659646053837


## I got 62% accuracy with hyperparameter tuning using GridSearchCV